# Correcting the map counts to account for diving seals and higher on-the-ground counts
This file includes the code that inflates the map counts using detection rate-estimating functions fitted with data from Erebus Bay, where ground counts could be conducted.  
  
In a nutshell, we use these models to estimate the detection rate, and then inflate the map counts according to this rate.


In [3]:
## Clear memory
rm(list=ls())
gc()

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658573,35.2,1168576,62.5,1168576,62.5
Vcells,1115063,8.6,4713750,36.0,5352274,40.9


### Loading libraries, functions and data

In [4]:
libs<-c("ggplot2","plyr","dplyr")
lapply(libs, require, character.only = TRUE)
pathToLocalGit<-"/home/ubuntu/Workspace/ContinentalWESEestimates/"

source(paste0(pathToLocalGit,"scripts/countSealsFromTags_functions.R"))

## Load the data - the data were generated from notebook: "Count Seals From Tags.ipynb"
load(file=paste0(pathToLocalGit,"data/estimatesByMap_unadjusted.RData"))
head(df)

## Print and check that this is the correct version of the data
estByRegionQ<-as.data.frame(mapcounts[,c("region","lowerH","estimateH","upperH")] %>% group_by(region) %>% dplyr::summarize(lower=ceiling(sum(lowerH)),estimate=ceiling(sum(estimateH)),upper=ceiling(sum(upperH))))
estByRegionQ<-rbind(estByRegionQ,data.frame(region="Total",lower=round(sum(mapcounts$lowerH)),estimate=round(sum(mapcounts$estimateH)),upper=round(sum(mapcounts$upperH))))
print(estByRegionQ)

[[1]]
[1] TRUE

[[2]]
[1] TRUE

[[3]]
[1] TRUE

,regionMapId,originHour,lclNumSeals,estNumSeals,uclNumSeals,mapcoords.x1,mapcoords.x2,acquisition_date,region,satId,⋯,originTZ,year,totalTags,scaledTotalTags,numHour,sinH,resid,estimateH,lowerH,upperH
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dttm>,<chr>,<chr>,⋯,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,AMU101663,21,1,1,4,-85.48336,-73.34752,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58817634,1,1,4
2,AMU102072,21,1,1,4,-85.44474,-73.41747,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,2,-0.29654759,0.1666667,0.8660254,-0.01598454,1,1,4
3,AMU102099,21,1,1,4,-85.79235,-73.17324,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,2,-0.29654759,0.1666667,0.8660254,-0.77273819,1,1,4
4,AMU103615,21,1,1,4,-85.79235,-73.17696,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58817634,1,1,4
5,AMU103695,21,1,1,4,-85.79235,-73.16951,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,1,-0.33940735,0.1666667,0.8660254,-0.58781338,1,1,4
6,AMU103791,21,1,6,14,-85.81166,-73.17324,2010-11-20 21:50:18,AMU,WV01,⋯,PST,2010,8,-0.03938903,0.1666667,0.8660254,3.12751275,6,1,14


  region lower estimate  upper
1    AMU  1013     2189   6223
2    EA1  2375     5967  16126
3    EA2  2174     5063  13934
4    QMA  2833     7142  19337
5    RSS  8330    22333  59682
6    WAP  3892     8894  22417
7  Total 20617    51588 137719


Now we use the handy function to request the detection rates under both models. Note that we can specify the "weight" of island counts. This is the proportion of map locations that resemble the islet haul-out locations in Erebus Bay (i.e., Turk's Head-Tryggve and Hutton Cliffs). According to Michelle and David, the vast majority of counts came from such locations. Here we are conservative and assume 95% of locations are islets.

In [12]:
names(mapcounts)<-gsub("scaledTotalTags","scaledNumTags",names(mapcounts))

adjRates<-predictDetRates(pathToGit=pathToLocalGit,dat=mapcounts,keyFieldName="regionMapId",islandWeight=0.95)
head(adjRates)

,regionMapId,wgtPredColRate,wgtPredIslRate,Year
,<chr>,<dbl>,<dbl>,<chr>
1,AMU101663,0.2933391,0.2348456,2010
2,AMU102072,0.2970266,0.2375183,2010
3,AMU102099,0.2970266,0.2375183,2010
4,AMU103615,0.2933391,0.2348456,2010
5,AMU103695,0.2933391,0.2348456,2010
6,AMU103791,0.3191522,0.2535545,2010


Finally, we add the detection rates to our data.frame of map counts, inflate, and summarize the results.  
  
Since detectionRate = count-in-map/count-on-the-ground, and we want count-on-the-ground, then:
Count-on-the-ground = count-in-map/detectionRate

In [ ]:
countdf<-merge(df,adjRates,by="regionMapId")

countdf$mdlColEstimate<-round(countdf$estimateH/countdf$wgtPredColRate)
countdf$mdlColUpper<-round(countdf$upperH/countdf$wgtPredColRate)
countdf$mdlColLower<-round(countdf$lowerH/countdf$wgtPredColRate)

countdf$mdlIslEstimate<-round(countdf$estimateH/countdf$wgtPredIslRate)
countdf$mdlIslUpper<-round(countdf$upperH/countdf$wgtPredIslRate)
countdf$mdlIslLower<-round(countdf$lowerH/countdf$wgtPredIslRate)

## Using the colony model:
print("Colony model")
estByRegionCol<-as.data.frame(countdf[,c("region","mdlColLower","mdlColEstimate","mdlColUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlColLower)),Estimate=round(sum(mdlColEstimate)),Upper=round(sum(mdlColUpper))))
estByRegionCol<-rbind(estByRegionCol,data.frame(region="Total",Lower=round(sum(countdf$mdlColLower)),Estimate=round(sum(countdf$mdlColEstimate)),Upper=round(sum(countdf$mdlColUpper))))
print(estByRegionCol)

In [ ]:
## Using the islet/mainland model:
print("Islet/mainland model")
estByRegionIsl<-as.data.frame(countdf[,c("region","mdlIslLower","mdlIslEstimate","mdlIslUpper")] %>% group_by(region) %>% dplyr::summarize(Lower=round(sum(mdlIslLower)),Estimate=round(sum(mdlIslEstimate)),Upper=round(sum(mdlIslUpper))))
estByRegionIsl<-rbind(estByRegionIsl,data.frame(region="Total",Lower=round(sum(countdf$mdlIslLower)),Estimate=round(sum(countdf$mdlIslEstimate)),Upper=round(sum(countdf$mdlIslUpper))))
print(estByRegionIsl)